In [1]:
import pandas as pd
import numpy as np

from average_conn_V2 import _simulate_img,_extract_time_series_voxel, _extract_time_series_parcels, _average_conn, _load_parcels_compute_mask, _load_voxel_data, _resample_mist, _compute_tseries, _build_size_roi, _compute_afc, _compute_brute_afc 

# Let's test the software first

Simulate some data with only one "spot", and calculate time series correltion

In [2]:
img, mask, corr = _simulate_img()
corr

0.6210103489634108

Brute force AFC calculation

In [3]:
time_series_voxel, masker_voxel = _extract_time_series_voxel(img, mask)

In [4]:
np.corrcoef(time_series_voxel.transpose())

array([[1.        , 0.62101035],
       [0.62101035, 1.        ]])

In [5]:
assert abs(np.corrcoef(time_series_voxel.transpose())[1, 0] - corr) < 1e-10

AFC the NIAK way

In [6]:
afc_test = _average_conn(img, mask, mask)
afc_test

array([[0.62101035]])

In [7]:
assert np.abs(corr - afc_test) < 1e-10

## It works on simple data

# Now for whole-brain intra-network average connectivity

We first get the mask of parcels, demo dataset, and build a group combined mask

In [8]:
img_mist, imgs, confounds, mask_combined = _load_parcels_compute_mask()

Optionally visualise the mask

In [9]:
#from nilearn.plotting import plot_roi
#plot_roi(mask_combined, resampling_interpolation='nearest')

Load fMRI data for one subject

In [10]:
img_n, time_series_voxel = _load_voxel_data(mask_combined, img_mist, imgs[0], confounds[0])

Resample mist to the data space

In [11]:
mist_resampled, mist_voxel = _resample_mist(img_mist, mask_combined)

Compute the time series using mist resampled

In [12]:
time_series_mist, masker_mist = _compute_tseries(mist_resampled, img_n)

Compute the AFC and also get the parcel sizes

In [13]:
afc, size_parcels = _compute_afc(time_series_mist, masker_mist, mist_voxel)

/Users/natashaclarke/ad_sz/scripts/average_conn_V2.py:94: RuntimeWarning: invalid value encountered in divide
  afc = ((size_parcels * size_parcels) * var_mist - size_parcels) / (size_parcels * (size_parcels - 1))


## Compute AFC by brute force for all parcels

In [14]:
brute_afc = _compute_brute_afc(time_series_voxel, mist_voxel, masker_mist)

/Users/natashaclarke/ad_sz/ad_sz_env/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/natashaclarke/ad_sz/ad_sz_env/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Compare the two methods

In [15]:
num_parcel = 1
print(f"parcel number {num_parcel}, size {size_parcels[num_parcel - 1]}: 'NIAK' AFC ={afc[num_parcel - 1]}, 'brute force' AFC ={brute_afc[num_parcel - 1]}")

parcel number 1, size [323.]: 'NIAK' AFC =[0.0690087], 'brute force' AFC =[0.10419664]


## So, they don't match. Suggestion from Hao-Ting: it might be the mask difference - test has a perfect overlapping mask and parcel, and real data does not (mask_combined, mist_resampled)